In [3]:
# from bs4 import BeautifulSoup as BS
# with open("nso.html", 'r') as file:
#     contents = file.read()

# soup = BS(contents, "html.parser")
# event_links = [x for link in soup.findAll('a') if (x:=link.get('href')).startswith('https://nso.upenn.edu/event/')]

# for event in event_links[0:1]:
#     x = BS(requests.get(event).text)
#     x.findAll('a')
    

In [4]:
# Fuck it we're using selenium now
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor
from ics import Calendar, Event
from bs4 import BeautifulSoup as BS
import functools
from typing import TypeAlias, Any
from enum import StrEnum

In [5]:
class Audience(StrEnum):
    FIRST_YEAR = "First-Year"
    SECOND_YEAR = "Second-Year"
    EXCHANGE_IGSP = "Exchange/IGSP"
    FGLI = "FGLI"
    INTERNATIONAL = "International"
    TRANSFER = "Transfer"
    ANY = "ANY"


class RawEvent:
    def __init__(self) -> None:
        self.audience: set[Audience]
        self.link: str
        self.cal_str: str
        self.loc_name: str | None
        self.loc_addr: str | None
        

In [6]:
# Set up firefox options
options = Options()
options.add_argument("--headless") # type: ignore

In [7]:
browser = webdriver.Firefox(options)
browser.get("https://nso.upenn.edu/events/events-calendar/")
assert "Events Calendar" in browser.title


In [8]:
# Find all the links to events
EventRecord: TypeAlias = tuple[str, list[Audience]]
event_links: list[EventRecord] = []

for event_anchor in browser.find_elements(By.XPATH, "//a [@rel='bookmark']"):
    # Event type is stored as an anchor element under the ancestor of the event link.
    event_type_urls = event_anchor.find_elements(
        By.XPATH, ".//ancestor::li//ul [@class='event-badges']/li/a"
    )  # type:ignore
    # Extract the intended audience from the tags
    event_type = {Audience(url.accessible_name) for url in event_type_urls} or {Audience.ANY}
    event_links.append((event_anchor.get_attribute('href'), event_type))
    # "//ul [@class='event-badges']"
event_links

[('https://nso.upenn.edu/event/breakfast-for-students-only/?sd=1692763200&ed=1000&ad',
  {<Audience.ANY: 'ANY'>}),
 ('https://nso.upenn.edu/event/exchange-igsp-check-in/?sd=1692763200&ed=1000&ad',
  {<Audience.EXCHANGE_IGSP: 'Exchange/IGSP'>}),
 ('https://nso.upenn.edu/event/exchange-igsp-welcome-session/?sd=1692763200&ed=1100&ad',
  {<Audience.EXCHANGE_IGSP: 'Exchange/IGSP'>}),
 ('https://nso.upenn.edu/event/resources-for-students-with-disabilities-2/?sd=1692763200&ed=1100&ad',
  {<Audience.EXCHANGE_IGSP: 'Exchange/IGSP'>,
   <Audience.FGLI: 'FGLI'>,
   <Audience.FIRST_YEAR: 'First-Year'>,
   <Audience.INTERNATIONAL: 'International'>,
   <Audience.TRANSFER: 'Transfer'>}),
 ('https://nso.upenn.edu/event/transfer-student-info-session-ii/?sd=1692763200&ed=1130&ad',
  {<Audience.TRANSFER: 'Transfer'>}),
 ('https://nso.upenn.edu/event/scavenger-hunt/?sd=1692763200&ed=1200&ad',
  {<Audience.INTERNATIONAL: 'International'>}),
 ('https://nso.upenn.edu/event/navigating-the-first-generation-and

In [24]:
Entry: TypeAlias = tuple[Event, list[Audience]]


def get_ics(record: EventRecord) -> Entry | None:
    link, audience = record
    audience = audience or [Audience.ANY]

    browser = webdriver.Firefox(options)
    browser.get(link)
    x = browser.find_element(By.XPATH, "//a [@id='apple-calendar-link']")
    cal_content = unquote(x.get_attribute("href")).split(",", 1)[
        1
    ]  # Remove the header info # type: ignore

    # Extract all calendar data.
    try:
        loc_name = browser.find_element(
            By.XPATH, "//div [@class='location-title']"
        ).text
    except NoSuchElementException:
        loc_name = "No Address provivided"

    try:
        loc_address = " ".join(
            browser.find_element(By.XPATH, "//div [@id='location-address']")
            .text.strip()
            .splitlines()
            # Remove prefix caused by screen reader text.
        ).removeprefix("Address for")
    except NoSuchElementException:
        loc_address = "No address provided"

    browser.close()
    location_details = f"{loc_name}: {loc_address}."

    cal_content = cal_content.splitlines()
    # Needed as ical library doesn't allow ommitting PROD ID
    cal_content.insert(1, "PRODID:NSO_CAL")

    # Cause whoever at NSO wrote this can't write ical's for shit and the end date is sometimes the epoch time.
    try:
        events = Calendar("\n".join(cal_content)).events
        event = events.pop()  # There should only be one event so this works
    except ValueError:
        print(f"Entry with url {link} has malformed date information")
        return
    except KeyError:
        print(f"Entry with url {link} did not contain any calendar entries")

    event.location = location_details
    event_description = event.description
    event.description = f"Intended audience: {' '.join(audience)}"
    if event_description is not None:
        extracted_description = BS(
            event_description, "html.parser"
        ).text.strip()  # For some reason we get unformatted html in the text. Thanks NSO
        event.description += f"\n\n{extracted_description}"
    event.url = link
    return (event, audience)


def collect_entries() -> list[Entry]:
    executor = ThreadPoolExecutor()
    return [x for x in executor.map(get_ics, event_links) if x is not None]

In [21]:
# DEBUG
# call = get_ics(("https://nso.upenn.edu/event/wellness-at-penn-session/?sd=1692763200&ed=1430&ad", None))
# call

In [25]:
# This takes a while, don't worry if it's taking a long time.
calendar = collect_entries()

Entry with url https://nso.upenn.edu/event/tphin-led-lunch/?sd=1692763200&ed=13&ad has malformed date information
Entry with url https://nso.upenn.edu/event/tphin-led-lunch-2/?sd=1692849600&ed=13&ad has malformed date information


[(<Event 'Breakfast For Students Only' begin:2023-08-23T11:30:00+00:00 end:2023-08-23T14:00:00+00:00>,
  {<Audience.ANY: 'ANY'>}),
 (<Event 'Exchange/IGSP Check-In' begin:2023-08-23T13:15:00+00:00 end:2023-08-23T14:00:00+00:00>,
  {<Audience.EXCHANGE_IGSP: 'Exchange/IGSP'>}),
 (<Event 'Exchange/IGSP Welcome Session' begin:2023-08-23T14:00:00+00:00 end:2023-08-23T15:00:00+00:00>,
  {<Audience.EXCHANGE_IGSP: 'Exchange/IGSP'>}),
 (<Event 'Resources For Students With Disabilities' begin:2023-08-23T14:00:00+00:00 end:2023-08-23T15:00:00+00:00>,
  {<Audience.EXCHANGE_IGSP: 'Exchange/IGSP'>,
   <Audience.FGLI: 'FGLI'>,
   <Audience.FIRST_YEAR: 'First-Year'>,
   <Audience.INTERNATIONAL: 'International'>,
   <Audience.TRANSFER: 'Transfer'>}),
 (<Event 'Transfer Student Info Session II' begin:2023-08-23T14:00:00+00:00 end:2023-08-23T15:30:00+00:00>,
  {<Audience.TRANSFER: 'Transfer'>}),
 (<Event 'Scavenger Hunt' begin:2023-08-23T14:00:00+00:00 end:2023-08-23T16:00:00+00:00>,
  {<Audience.INTERNA

In [39]:

def __reduction_function(cal: Calendar, event: tuple[Event, Any]):
    cal.events.add(event[0])
    return cal

def reduce_cal(cal_list: list[Entry]) -> Calendar:
    # This order matters for some reason, normally could have Calendar() as the initial parameter in the fold.
    calendar = Calendar()
    calendar.method = "REQUEST"
    calendar = functools.reduce(__reduction_function, iter(cal_list), calendar)
    return calendar

def create_cal(cal_list: list[Entry], predicates: set[Audience]) -> Calendar:
    events = [entry for entry in cal_list if predicates.intersection(entry[1])]
    return reduce_cal(events)

In [40]:
# Calendar with all events
general_calendar = reduce_cal(calendar)
with open("./output_calendars/general_calendar.ics", "w") as file:
    file.writelines(general_calendar.serialize_iter())

In [43]:
# Calendar for exchange students only
exchange_calendar = create_cal(calendar, {Audience.EXCHANGE_IGSP, Audience.ANY})
with open("./output_calendars/exchange_igsp_events.ics", 'w') as file:
    file.writelines(exchange_calendar.serialize_iter())

In [42]:
# Scratch space to figure out how the hell they broke the date time for some events so badly

# import datetime
#
# iterator = enumerate(cal_content.splitlines())
# start_index, start_time = next(x for x in iterator if x[1].startswith("DTSTART:"))
# end_index, broken_time = next(x for x in iterator if x[1].startswith("DTEND:"))
#
# print(start_time)
# dt_start_time = datetime.datetime.fromisoformat(start_time.split(":", 1)[1])
# print(broken_time)
# time = browser.find_element(By.XPATH, "//*[@id='single-events-top']/div[1]/p[2]").text
#
# split = time.splitlines()
# parsed_start = datetime.datetime.strptime(
#     split[1].rstrip().removesuffix("-").strip(), "%I:%M %p"
# ).time()
# parsed_end = datetime.datetime.strptime(split[-1].strip(), "%I:%M %p").time()
#
# parsed_start, parsed_end